In [ ]:
from spks import *
# from pathlib import Path
folder = Path('/scratch/ks25_sorting_20230905_131941_tr9v')

with open(folder/'params.py','r') as f:
    params = f.read()
params = params.split('\n')
for i,p in enumerate(params):
    if p.startswith('dat_path'):
        params[i] = "dat_path = 'filtered_recording.ap.bin'"
with open(folder/'params.py','w') as f:
    f.write("\n".join(params))


In [ ]:
%matplotlib widget

from spks import *



    # class timestamps():
    #     def __init__(self)

folder = Path('/scratch/ks25_sorting_20230905_131941_tr9v')

sp = Clusters(folder)


In [ ]:
def plot_footprints(waves, channel_xy, gain = (10,0.01),shade_data = None,shade_color = 'k',shade_alpha=0.3,
                                srate = 30000, plotscale=None, ax = None,**kwargs):
    '''
    Plots multichannel waveforms 
    
    
    Inputs:
    -------
        - waves (nspikes x nsamples x nchannels) or (nsamples x nchannels)
        - channel_xy positions (nchannels x 2)
        - gain (2d tuple)
        - srate (for the scale)
        - plotscale: True to draw the scale
    '''
    import pylab as plt
    
    p = []
    if ax is None:
        ax = plt.gca()
    if len(waves.shape) == 2:
        # plotting mean waveforms (these must be average waveforms)
        waves = waves.copy().reshape(1,*waves.shape)
    wtime = (1000.*(np.arange(waves.shape[1])-waves.shape[1]/2)/srate).astype(np.float32)
    for iw,wv in enumerate(waves):
        for i in range(wv.shape[1]):
            # plot each channel
            p.append(ax.plot(wtime*gain[0] + channel_xy[i,0],
                              wv[:,i]*gain[1]+channel_xy[i,1],
                             clip_on=False,**kwargs))
            if not shade_data is None:
                ax.fill_between(wtime*gain[0] + channel_xy[i,0],
                              (wv[:,i]+shade_data[iw,:,i])*gain[1]+channel_xy[i,1],
                              (wv[:,i]-shade_data[iw,:,i])*gain[1]+channel_xy[i,1],alpha = shade_alpha,
                              edgecolor = None,
                              facecolor=shade_color)
        miny = np.min(abs(np.diff(channel_xy[:,1])))
        ax.axis([np.min(channel_xy[:,0]),
                 np.max(channel_xy[:,0])+wtime[-1]*gain[0],
                np.min(channel_xy[:,1])-miny,
                np.max(channel_xy[:,1])+miny])
        ax.axis('off')
    if not plotscale is None:
        t = []
        # X scale
        x = plotscale[0] * np.array([0,1])*gain[0] - gain[0]/6.
        y = np.array([0,0])+np.min(channel_xy[i,0]) - gain[1]/2.

        p.append(plt.plot(x,y,'k',clip_on = False,lw = 1))
        t.append(ax.text(np.diff(x)/2. + np.min(x),np.min(y),
                r'{0} ms'.format(plotscale[0]),
                va = 'top',ha='center',fontsize=8))
        # Y scale
        x = np.array([0,0])-gain[0]/6.
        y = np.array([0,plotscale[1]*gain[1]])+np.min(channel_xy[i]) - gain[1]/2.
        p.append(plt.plot(x,y,'k',clip_on = False,lw = 1))
        t.append(ax.text(np.min(x),np.diff(y)/2. + np.min(y),
                r'{0} $\mu$V'.format(plotscale[1]),
                rotation = 90,va = 'center',ha='right',fontsize=8))
        return p,t
    return p


In [ ]:
import matplotlib.style as mplstyle
mplstyle.use('fast')

# icluster = 20
# wave = sp.get_cluster_waveforms(icluster,n_waveforms=50)
# plot_footprints(wave,sp.channel_positions,gain=[10,0.05],lw = 0.1,color='k');
# plot_footprints(sp.cluster_waveforms_mean[sp.unique_clusters==icluster],sp.channel_positions,gain=[10,0.05],color='r',lw=1);
sp.cluster_waveforms_std[1][:,electrodes[1]]

In [ ]:
icluster = 0

fig = plt.figure()
ax_waves = fig.add_axes([0.1,0.1,0.8,0.8])
cluster_positions,principal_channel = waveforms_position(sp.cluster_waveforms_mean,sp.channel_positions)
electrodes = []
for p in principal_channel:
    electrodes.append(np.where(np.array([np.linalg.norm(c - sp.channel_positions[p]) for c in sp.channel_positions])<100)[0])

def plot_func(icluster = 20):
    icluster = int(icluster)
    clu_num = sp.unique_clusters[icluster]
    ax_waves.cla()
    ax_waves.plot(sp.channel_positions[:,0],sp.channel_positions[:,1],'kx',markersize = 3,alpha = 0.5)
  
    # wave = sp.get_cluster_waveforms(icluster,n_waveforms=50)
    # plot_footprints(wave[:,:,electrodes[icluster]],sp.channel_positions[electrodes[icluster]],gain=[10,0.05],lw = 0.1,color='k');
    plot_footprints(sp.cluster_waveforms_mean[clu_num][:,electrodes[clu_num]],
                    sp.channel_positions[electrodes[clu_num]],gain=[10,0.05],color='r',lw=1,ax = ax_waves);
    plot_footprints(sp.cluster_waveforms_mean[clu_num][:,electrodes[clu_num]],
                    sp.channel_positions[electrodes[clu_num]],
                    shade_data = sp.cluster_waveforms_std[clu_num][:,electrodes[clu_num]],
                    gain=[10,0.05],color='k',lw=1,ax = ax_waves);
    ax_waves.plot(sp.channel_positions[principal_channel[clu_num],0],
    sp.channel_positions[principal_channel[clu_num],1],'ro',markersize = 5,alpha = 0.5)
    wtime = (1000.*(np.arange(sp.cluster_waveforms_mean.shape[1])-sp.cluster_waveforms_mean.shape[1]/2)/30000).astype(np.float32)

    ax_waves.plot(wtime*10 + sp.channel_positions[principal_channel[clu_num],0],
                sp.cluster_waveforms_mean[clu_num][:,principal_channel[clu_num]].squeeze()*0.05+sp.channel_positions[principal_channel[clu_num],1],
                             clip_on=False,color = 'b')
    ax_waves.axis('tight')
plot_func(icluster=20)


ax_select = fig.add_axes([0.25, 0.1, 0.65, 0.03])
from matplotlib.widgets import Slider
sel_slider = Slider(
    ax=ax_select,
    label='cluster',
    valmin=0,
    valmax=len(sp)-1,
    valinit=20,
)
sel_slider.on_changed(plot_func)

def on_press(event):
    if event.key == 'right':
        icluster = np.clip(int(sel_slider.val) + 1,0,len(sp)-1)
        sel_slider.set_val(icluster)
    elif event.key == 'left':
        icluster = np.clip(int(sel_slider.val) - 1,0,len(sp)-1)
        sel_slider.set_val(icluster)

fig.canvas.mpl_connect('key_press_event', on_press)

In [ ]:
sp.unique_clusters == 96

In [ ]:
plot_func(234)

In [ ]:
waveforms = sp.cluster_waveforms_mean
channel_positions = sp.channel_positions

peak_to_peak = (waveforms.max(axis = 1) - waveforms.min(axis = 1))
# the amplitude of each waveform is the max of the peak difference for all channels
amplitude = np.abs(peak_to_peak).max(axis=1)
# compute the center of mass (X,Y) of the waveforms
centerofmass = [peak_to_peak*pos for pos in channel_positions.T]
centerofmass = np.vstack([np.sum(t,axis =1 )/np.sum(peak_to_peak,axis = 1) 
                                    for t in centerofmass]).T
# the peak channel is the absolute max of the peak_to_peak
peak_channels = np.argmax(np.abs(peak_to_peak),axis = 1)
print(peak_to_peak.shape,waveforms.shape,peak_channels.shape)

In [ ]:
plt.figure()
plt.plot(peak_to_peak[234])
peak_channels = np.argmax(np.max(np.abs(waveforms),axis=1),axis=1)
# peak_channels = np.argmax(np.abs(peak_to_peak),axis = 1)
plt.plot(peak_channels[234],peak_to_peak[234][peak_channels[234]],'ro')
channel_positions[peak_channels[234]]

In [ ]:
plt.figure()
maxim = waveforms.max(axis=1)
pc2 = np.argmax(np.abs(waveforms).max(axis=1),axis=1)
imaxim = waveforms.argmax(axis=1)
minim = waveforms.min(axis=1)
iminim = waveforms.argmin(axis=1)
icluster = 144
# mm = maxim[icluster][iw]-minim[icluster][iw]
pc = np.argmax(maxim-minim,axis=1)
for iw in range(waveforms[icluster].shape[1]):
    x,y = channel_positions[iw]
    plt.plot(x+0.4*np.arange(waveforms.shape[1]),waveforms[icluster][:,iw]*0.1+y,lw=0.5,color='k');
    plt.plot(x+0.4*imaxim[icluster][iw],maxim[icluster][iw]*0.1+y,'ro',markersize = 1);
    plt.plot(x+0.4*iminim[icluster][iw],minim[icluster][iw]*0.1+y,'bo',markersize = 1);
    col='k'
    if iw == pc2[icluster]:
        col = 'r'
    if iw == principal_channel[icluster]:
        col = 'g'

    plt.text(x+0.4*30,y,'{0:.0f}'.format(maxim[icluster][iw]-minim[icluster][iw]),fontsize=5,color = col);

In [ ]:
imaxim.shape

In [ ]:

# folder
dat2 = map_binary(folder/'filtered_recording.ap.bin',meta['nchannels'])
meta = load_dict_from_h5(folder/'filtered_recording.ap.metadata.hdf')
meta['file_offsets'],dat.file_sample_offsets,dat.shape,dat2.shape

In [ ]:

folder = Path('/scratch/ks25_sorting_20230905_131941_tr9v')
sessionfiles = ['/home/data/JC131/20230901_113844/ephys_g0/ephys_g0_imec0/ephys_g0_t0.imec0.ap.bin',
                '/home/data/JC131/20230901_115632/ephys_g1/ephys_g1_imec0/ephys_g1_t0.imec0.ap.bin']

dat = RawRecording(sessionfiles,return_preprocessed = False)

meta = load_dict_from_h5(folder/'filtered_recording.ap.metadata.hdf')
dat2 = map_binary(folder/'filtered_recording.ap.bin',meta['nchannels'])

sp = Clusters(folder)

mwaves = extract_waveform_set(spike_times = sp,data = dat2,chmap = dat.channel_info.channel_idx.values,max_n_spikes=1000, chunksize=10)

In [ ]:
waveforms = {}
for iclu,w in zip(sp.unique_clusters,mwaves):
    waveforms[iclu] = w

import h5py


save_dict_to_h5(folder/'cluster_waveforms.hdf',waveforms)


In [ ]:


#waveforms = load_dict_from_h5(folder/'cluster_waveforms.hdf')

import h5py as h5
waveforms_file = h5.File(folder/'cluster_waveforms.hdf','r')

In [ ]:



from multiprocessing.pool import Pool
from multiprocessing import cpu_count

def _mean_std_from_cluster_waveforms(icluster,folder):
    import h5py as h5
    with  h5.File(folder/'cluster_waveforms.hdf','r') as waveforms_file:
        mwave = np.mean(waveforms_file[str(icluster)][()],axis = 0) 
        stdwave = np.std(waveforms_file[str(icluster)][()],axis = 0)/waveforms_file[str(icluster)].shape[0]
    return np.stack([mwave,stdwave])

with Pool(12) as pool:
    result = [r for r in tqdm(pool.imap(partial(_mean_std_from_cluster_waveforms,folder = folder),sp.unique_clusters),
                    desc='Computing mean waveforms', total = len(sp))]
    meanwaves = np.stack([r[0] for r in result])
    sterrwaves = np.stack([r[1] for r in result])
    
    
    
    
    


In [ ]:
# %matplotlib widget 
import pylab as plt
from spks import *

plt.figure()


iclus = 200
for mw in mwaves[iclus][:20]:
    plot_footprints(waves = ,
                    channel_xy = np.stack(dat.channel_info.channel_coord.values), gain=[15,0.07],lw = 0.1);

plot_footprints(waves = mwaves[iclus].mean(axis=0),
                    channel_xy = np.stack(dat.channel_info.channel_coord.values), gain=[15,0.07],lw = 1,color = 'r');

In [ ]:

pos, peak = waveforms_position(sp.templates_raw,sp.channel_positions)
peak_to_peak = (sp.templates_raw.max(axis = 1) - sp.templates_raw.min(axis = 1)).max(axis=1)

###
import pylab as plt
%matplotlib widget
plt.figure()
plt.plot(sp.channel_positions[:,0],sp.channel_positions[:,1],'o',color='lightgray')
plt.scatter(pos[20,0],pos[20,1],30,peak_to_peak[20],alpha = 0.5,cmap='hot')
plt.colorbar()
plt.plot(sp.channel_positions[peak[20],0],sp.channel_positions[peak[20],1],'x')
from spks.viz import plot_footprints
plot_footprints(sp.templates_raw[20],sp.channel_positions,gain=[5,0.3]);
plt.show()

In [ ]:
sp['unit selection','probe','shank','unit']

In [ ]:
spike_clusters = sp.clusters
spike_times = sp.spike_times


In [ ]:
sp.templates_raw.shape

In [ ]:


get_overlapping_spikes_indices(sp.spike_times,sp.clusters, sp.templates_raw, sp.channel_positions)

In [ ]:
list(folder.glob("*"))

In [ ]:
from spks.sorting import run_ks25
sessionfiles = ['/home/data/JC131/20230901_113844/ephys_g0/ephys_g0_imec0/ephys_g0_t0.imec0.ap.bin',
                   '/home/data/JC131/20230901_115632/ephys_g1/ephys_g1_imec0/ephys_g1_t0.imec0.ap.bin']

run_ks25(sessionfiles)
